<a href="https://colab.research.google.com/github/AnnaZhuravleva/HSE/blob/master/course%204/sem03/Sem03_semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

In [3]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors
import gensim
import zipfile
import urllib.request
import numpy as np
import pandas as pd
import nltk
import pymorphy2
from scipy import spatial
import re
import os
import tensorflow as tf
import sys

from google.colab import drive
import time
from time import time

pmm = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
drive.mount('/content/drive')
elmo_path = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/elmo'
project_path = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/'
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **FastText Search**



In [0]:
# urllib.request.urlretrieve("http://vectors.nlpl.eu/repository/11/181.zip", "/content/drive/My Drive/Colab Notebooks/infosearch/181.zip")
# z = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/181.zip', 'r')
# z.extractall('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/')
collection = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/collection.csv'
model = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model'
corpus = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/quora_question_pairs_rus.csv'

In [0]:
class FastTextSearch:

    def __init__(self, model, collection, corpus):
        self.model = gensim.models.KeyedVectors.load(model)
        self.collection = pd.read_csv(collection) # возьмем тот же файл, что и в прошлой домашке
        self.corpus = self.collection['question2']
        self.questions = pd.read_csv(corpus)['question1']
        self.test_corpus = self.test_data()
        self.matrix = self.index_fasttext()
        
    def fasttext_search(self, query):

      lemmas_vectors = np.zeros((len(query), self.model.vector_size))
      vec = np.zeros((self.model.vector_size,))

      for idx, lemma in enumerate(query):
        if lemma in self.model.vocab:
          lemmas_vectors[idx] = self.model.wv[lemma]
        else:
          pass

      if lemmas_vectors.shape[0] is not 0:
          vec = np.mean(lemmas_vectors, axis=0)
          
      return vec

    def index_fasttext(self):
        matrix_fasttext = np.zeros((self.corpus.shape[0], self.model.vector_size))
        start_time = time()
        for row, query in enumerate(self.corpus):  
          vector = self.fasttext_search(query.split()) # наш корпус уже нормализован
          for idx, cell in enumerate(vector):
              matrix_fasttext[row][idx] = cell
        print(f'Indexing FastTextSearch takes {time() - start_time} sec')
        return matrix_fasttext

    def simple_preproc(self, query):
        return [pmm.normal_forms(word)[0] for word in nltk.word_tokenize(query)]
        
    def test_data(self):
        test_data = []
        for question in self.questions[:3]:
            question = self.simple_preproc(question)
            vector = self.fasttext_search(question)
            test_data.append(vector)
        return test_data

    def testing(self, n):
        vec = self.test_corpus[0]
        res = {}
        for i in range(n):
          row = self.matrix[i]
          cos_sim = spatial.distance.cosine(row, vec)
          res[i] = cos_sim

        res = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
        return res

In [0]:
fts = FastTextSearch(model=model, corpus=corpus, collection=collection)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Indexing FastTextSearch takes 73.68462038040161 sec


In [0]:
fts.testing()

[(75, 0.826882691091914),
 (1206, 0.8187299962122366),
 (547, 0.8023287962484263),
 (608, 0.7918055085531388),
 (1929, 0.7889931007593103)]

# **ELMO**

In [0]:
import tensorflow as tf
from elmo_helpers import tokenize, get_elmo_vectors, load_elmo_embeddings

In [0]:
class ElmoSearch:
  
  def __init__(self):
    tf.reset_default_graph()
    self.batcher, self.sentence_character_ids, self.elmo_sentence_input = \
    load_elmo_embeddings(elmo_path)
    self.vectors = []
    
  def build_vec(self, sentences):
     with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        elmo_vectors = get_elmo_vectors(
            sess, sentences, self.batcher, self.sentence_character_ids,
            self.elmo_sentence_input)
        
        print(f'Tensor shape: {elmo_vectors.shape}')
        results = []
        for vect, sent in zip(elmo_vectors, sentences):
          results.append(np.mean(vect[:len(sent), :], axis=0))
        
        return results
    
  def indexing(self, sentences):
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      start = time()
      sentences = [tokenize(sent) for sent in sentences]
      for sent in sentences:
        sent_vec = self.build_vec([sent])
        self.vectors.append(sent_vec[0])
        
      print(f'=====\n'\
            f'ElmoSearch Indexing takes {time() - start} sec'\
            f'for {len(sentences)} docs')
      return self.vectors
    
  def simple_preproc(self, query):
      return [pmm.normal_forms(word)[0] for word in nltk.word_tokenize(query)]
    
  def elmo_search(self, query):
    query = self.simple_preproc(query)
    query_vec = self.build_vec([query])
    
      
      
    
      
    

In [111]:
np.mean([[1,2,3], [4,5,6], [7,8,9]], axis=0)

array([4., 5., 6.])

In [112]:
els = ElmoSearch()
els.indexing(pd.read_csv(collection)['question2'][:2])
els.elmo_search('зарплаты Индии')

Sentences in this batch: 1


Tensor shape: (1, 12, 1024)


Sentences in this batch: 1


Tensor shape: (1, 8, 1024)
=====
ElmoSearch Indexing takes 4.347093105316162 secfor 2 docs


Sentences in this batch: 1


Tensor shape: (1, 2, 1024)


In [102]:
len(tmp2[0])

1024

## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   